<a href="https://colab.research.google.com/github/BeyersNel/BeyersNel.github.io/blob/master/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

from pylab import rcParams
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sns.set_style('whitegrid')

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn import metrics 
from sklearn.metrics import log_loss, classification_report, make_scorer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

import nltk
from nltk import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize, TreebankWordTokenizer

import string

from datetime import datetime

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [0]:
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

In [0]:
train.shape

(95662, 16)

In [0]:
test.shape

(45019, 15)

In [0]:
train.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

In [0]:
# train_fraud = train[train['FraudResult'] == 1]
# train_regular = train[train['FraudResult'] == 0]
# regular_pick = np.random.choice(train_regular.index, 90000, replace=False)
# train_regular_subset = train_regular.drop(regular_pick)
# train_regular_subset.shape

In [0]:
# train_regular_subset = train_regular_subset.reset_index(drop=True)
# train_fraud = train_fraud.reset_index(drop=True)
test = test.reset_index(drop=True)

In [0]:
# train = pd.concat([train_regular_subset, train_fraud], sort=False, axis=0)
train = pd.concat([train, test], sort=False, axis=0)

In [0]:
train.shape

(140681, 16)

In [0]:
def getTimeData(ps):
    df = pd.DataFrame()
    df['time'] = ps.apply(lambda x: datetime.strptime(x.split(':')[0],'%Y-%m-%dT%H'))
    df['weekday'] = df['time'].apply(lambda x: x.weekday())
    df['year'] = df['time'].apply(lambda x: x.year)
    df['month'] = df['time'].apply(lambda x: x.month)
    df['hour'] = df['time'].apply(lambda x: x.hour)
    return df.drop('time', axis=1);

In [0]:
df = pd.concat([train, getTimeData(train['TransactionStartTime'])], sort=False, axis=1)
df = df.drop('TransactionStartTime', axis=1)

In [0]:
TransactionId = df['TransactionId']
BatchId = df['BatchId']
AccountId = df['AccountId']
SubscriptionId = df['SubscriptionId']
CustomerId = df['CustomerId']

df = df.drop('TransactionId', axis=1)
df = df.drop('BatchId', axis=1)
df = df.drop('AccountId', axis=1)
df = df.drop('SubscriptionId', axis=1)
df = df.drop('CustomerId', axis=1)

In [0]:
df_dummies = pd.get_dummies(df)

In [0]:
y = df_dummies['FraudResult']
X = df_dummies.drop(['FraudResult'], axis=1)

In [0]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X[:-len(test)]
y_train = y[:-len(test)]
X_test = X[-len(test):]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
# y_test = np.array(y_test)

In [0]:
# [correctly false, incorrectly true]
# [incorrectly false, correctly true]

In [0]:
# parameters
# clf = GridSearchCV(SVC(), parameters, cv=3, iid=True)

In [0]:
from sklearn.svm import SVC

In [0]:
def doLogReg(X_train, y_train, X_test, C, solver):
    logreg = LogisticRegression(C=C, solver=solver, random_state=42)
    logreg.fit(X_train, y_train)
    return logreg.predict(X_test)

In [0]:
def runLogReg(X_train, y_train, X_test):
    results = []
    C = 0.0001
    for i in list(range(0,4)):
        for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
            results.append(doLogReg(X_train, y_train, X_test, C=C, solver=solver))
        C = C*100
    return results

In [0]:
def doSVC(X_train, y_train, X_test, C, kernel):
    clf = SVC(gamma='auto', C=C, kernel=kernel)
    clf.fit(X_train, y_train)
    return clf.predict(X_test)

In [0]:
def runSVC(X_train, y_train, X_test):
    results = []
    C = 0.0001
    for i in list(range(0,4)):
        for solver in ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']:
            print(f'{solver} {C}')
            results.append(doSVC(X_train, y_train, X_test, C=C, kernel=solver))
        C = C*100
    return results

In [0]:
import warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge. Increase the number of iterations.")
warnings.filterwarnings("ignore", message="The max_iter was reached which means the coef_ did not converge")
warnings.filterwarnings("ignore", message="The line search algorithm did not converge")
warnings.filterwarnings("ignore", message="Line Search failed")
warnings.filterwarnings("ignore", message="Rounding errors prevent the line search from converging")
warnings.filterwarnings("ignore", message="newton-cg failed to converge. Increase the number of iterations.")

In [0]:
# logReg_results = runLogReg(X_train, y_train, X_test)

In [0]:
# results_sum = np.array(logReg_results).sum(axis=0)
# np.place(results_sum, results_sum > 1, 1)

In [0]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
svc_pred = clf.predict(X_test)

In [0]:
# combined = svc_pred + results_sum
# np.place(combined, combined > 1, 1)

In [0]:
# from sklearn.metrics import f1_score
# f1_score(y_test, svc_pred, average='macro')

In [0]:
# confusion_matrix(y_test, svc_pred)

In [0]:
df = pd.DataFrame(test['TransactionId'])
df['FraudResult'] = svc_pred.astype(int)
df.to_csv('./submission.csv', index=False)

In [0]:
df.to_csv('./submission.csv', index=False)

In [0]:
# confusion_matrix(y_test, logReg_results)

In [0]:
# def logLossScorer(y_true, y_pred):
#     X = confusion_matrix(y_test, logreg_y_pred)
#     return log_loss(y_true, y_pred)

# cross_val_score(regressor, X, y, cv=5, scoring=scorer).mean()